# <center> CCT College Dublin </center>

## <center> Assessment Cover Page</center>

<p style="font-size:larger; text-align:center; color:red;"><em>
 To be provided separately as a word doc for students to include with every submission
</em></p>

<br><br><br>
---

<span style="font-size:larger;">

**Module Title:**&nbsp;&nbsp;&nbsp;Data Preparation
	
**Assessment Title:**&nbsp;&nbsp;&nbsp;Machine Learning (10 ETCS)
	
**Lecturer Name:**&nbsp;&nbsp;&nbsp;Dr. Muhammad Iqbal
	
**Student Full Name:**&nbsp;&nbsp;&nbsp;Yumiko Maria Bejarano Azogue 
	
**Student Number:**&nbsp;&nbsp;&nbsp;2024144
	
**Assessment Due Date:**&nbsp;&nbsp;&nbsp;21st April 2024
	
**Date of Submission:**&nbsp;&nbsp;&nbsp;21st April 2024
    
</span> 

<br><br><br>

---

#### Declaration 

```
By submitting this assessment, I confirm that I have read the CCT policy on Academic Misconduct and understand the implications of submitting work that is not my own or does not appropriately reference material taken from a third party or other source. I declare it to be my own work and that all material from third parties has been appropriately referenced. I further confirm that this work has not previously been submitted for assessment by myself or someone else in CCT College Dublin or any other higher education institution.
```
<br><br><br>


In [2]:
# import the pandas the library
import pandas as pd

In [4]:
# Import thi slibrary to suppress the warnings
import warnings
warnings.filterwarnings('ignore')   # The object 'warnings' is used to call the method 'filterwarnings' and ignore the warnings

In [5]:
# import the filee
df_collegePlace = pd.read_csv('collegePlace.csv')

In [6]:
# Display first 5 records
df_collegePlace.head()

,Age,Gender,Stream,Internships,CGPA,Hostel,HistoryOfBacklogs,PlacedOrNot
0,22,Male,Electronics And Communication,1,8,1,1,1
1,21,Female,Computer Science,0,7,1,1,1
2,22,Female,Information Technology,1,6,0,0,1
3,21,Male,Information Technology,0,8,0,1,1
4,22,Male,Mechanical,0,8,1,0,1
